In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torchvision.datasets import CIFAR10
import pytorch_lightning as pl

In [2]:
def seed_everything(seed=42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

class CReLU(nn.Module):
    def __init__(self):
        super(CReLU, self).__init__()
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.relu
        out = torch.cat((out(x), out(-x)), dim=1)
        return out

class PGLU(nn.Module):
    def __init__(self, channels_in, channels_out, kernel_size=1, padding=0, pool=False):
        super(PGLU, self).__init__()
        self.conv = nn.Conv2d(channels_in, channels_out, kernel_size, padding=padding)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(2, 2) if pool else None

    def forward(self, gate, output):
        gate = self.conv(gate)
        if self.pool is not None:
            gate = self.pool(gate)
        gate = self.relu(gate)
        gate = gate > 0
        return output * gate

class OGLU(nn.Module):
    def __init__(self, channels, kernel_size=1, padding=0):
        super(OGLU, self).__init__()
        self.conv = nn.Conv2d(channels, channels, kernel_size, padding=padding)
        self.relu = nn.ReLU()

    def forward(self, output):
        gate = self.conv(output)
        gate = self.relu(gate)
        gate = gate > 0
        return output * gate

class APGLU(nn.Module):
    def __init__(self, channels_in, channels_out, kernel_size=1, padding=0, pool=False):
        super(APGLU, self).__init__()
        self.conv = nn.Conv2d(channels_in, channels_out, kernel_size, padding=padding)
        self.sigmoid = nn.Sigmoid()
        self.pool = nn.MaxPool2d(2, 2) if pool else None

    def forward(self, gate, output):
        gate = self.conv(gate)
        if self.pool is not None:
            gate = self.pool(gate)
        gate = self.sigmoid(gate)
        return output * gate

class AOGLU(nn.Module):
    def __init__(self, channels, kernel_size=1, padding=0):
        super(AOGLU, self).__init__()
        self.conv = nn.Conv2d(channels, channels, kernel_size, padding=padding)
        self.sigmoid = nn.Sigmoid()

    def forward(self, output):
        gate = self.conv(output)
        gate = self.sigmoid(gate)
        return output * gate

In [3]:
class Net(pl.LightningModule):
    def __init__(self, lr=3e-4,
                 act=[nn.ReLU(), nn.ReLU(), nn.ReLU(), nn.ReLU()]):
        super(Net, self).__init__()
        self.lr = lr

        self.conv1 = nn.Conv2d(3, 32, 5, padding=2)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.bn1 = nn.BatchNorm2d(32)
        self.act1 = act[0]
        self.d1 = nn.Dropout(0.2)

        self.conv2 = nn.Conv2d(32, 64, 5, padding=2)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.bn2 = nn.BatchNorm2d(64)
        self.act2 = act[1]
        self.d2 = nn.Dropout(0.2)

        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.pool3 = nn.MaxPool2d(2, 2)
        self.bn3 = nn.BatchNorm2d(128)
        self.act3 = act[2]
        self.d3 = nn.Dropout(0.2)

        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.pool4 = nn.MaxPool2d(2, 2)
        self.bn4 = nn.BatchNorm2d(256)
        self.act4 = act[3]
        self.d4 = nn.Dropout(0.2)

        self.fc1 = nn.Linear(256*2*2, 10)

    def forward(self, x):
        out = self.conv1(x)
        out = self.pool1(out)
        out = self.bn1(out)
        if isinstance(self.act1, PGLU) or isinstance(self.act1, APGLU):
            out = self.act1(x, out)
        else:
            out = self.act1(out)
        out2 = self.d1(out)

        out = self.conv2(out2)
        out = self.pool2(out)
        out = self.bn2(out)
        if isinstance(self.act2, PGLU) or isinstance(self.act2, APGLU):
            out = self.act2(out2, out)
        else:
            out = self.act2(out)
        out3 = self.d2(out)

        out = self.conv3(out3)
        out = self.pool3(out)
        out = self.bn3(out)
        if isinstance(self.act3, PGLU) or isinstance(self.act3, APGLU):
            out = self.act3(out3, out)
        else:
            out = self.act3(out)
        out4 = self.d3(out)

        out = self.conv4(out4)
        out = self.pool4(out)
        out = self.bn4(out)
        if isinstance(self.act4, PGLU) or isinstance(self.act4, APGLU):
            out = self.act4(out4, out)
        else:
            out = self.act4(out)
        out = self.d4(out)

        out = out.view(-1, 256*2*2)
        out = self.fc1(out)
        out = self.d1(out)
        return out

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=self.lr)

    def train_dataloader(self):
        return DataLoader(CIFAR10(root='./data', train=True, download=True, transform=transforms.ToTensor()), batch_size=25000, shuffle=True)

    def val_dataloader(self):
        return DataLoader(CIFAR10(root='./data', train=False, download=True, transform=transforms.ToTensor()), batch_size=10000, shuffle=False)

    def test_dataloader(self):
        return DataLoader(CIFAR10(root='./data', train=False, download=True, transform=transforms.ToTensor()), batch_size=10000, shuffle=False)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        self.log('val_loss', loss, prog_bar=True)
        accuracy = (y_hat.argmax(dim=1) == y).float().mean()
        self.log('val_acc', accuracy, prog_bar=True)
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        return loss

In [ ]:
seed_everything(42)
net = Net(act=[OGLU(32), OGLU(64), OGLU(128), OGLU(256)])
trainer = pl.Trainer(accelerator='gpu', max_epochs=256, accumulate_grad_batches=2, enable_progress_bar=False)
trainer.fit(net)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3090 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name  | Type        | Params
---------------------------------------
0  | conv1 | Conv2d      | 2.4 K 
1  | pool1 | MaxPool2d   | 0     
2  | bn1   | BatchNorm2d | 64    
3  | act1  | OGLU        | 1.1 K 
4  | d1    | Dropout     | 0     
5  | conv2 | Conv2d      | 51.3 K
6  | pool2 | MaxPool2d   | 0     
7  | bn2   | BatchNorm2d | 128   
8  | act2  | OGLU        | 4.2 K 
9  | d2    | Dropout     | 0     
10 | conv3 | Conv2d     

Files already downloaded and verified


E:\Blake Projects\Transformer-Experiments\venv\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 24 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Files already downloaded and verified


E:\Blake Projects\Transformer-Experiments\venv\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 24 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
E:\Blake Projects\Transformer-Experiments\venv\lib\site-packages\pytorch_lightning\loops\fit_loop.py:280: PossibleUserWarning: The number of training batches (2) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


In [ ]:
max_accuracy = trainer.callback_metrics['val_acc'].max()
print(max_accuracy)